# HV19.16 B0rked Calculator

*Santa has coded a simple project for you, but sadly he removed all the operations. But when you restore them it will print the flag!*

We get an `.exe` file - thankfully it runs fine under [Wine](https://www.winehq.org/), as I prefer to work on Linux without a Windows VM when possible:

![Calculator screenshot](./calc.png)

Other than the "Back to basic" nightmare earlier this Hackvent, this was the first time I reverse-engineered a binary. I already had [Ghidra](https://ghidra-sre.org/) installed on my machine, but never used it so far. So I opened the exe in Ghidra to take a closer look!

At address `0040145b` (in what's probably the `main` function), we find (slightly edited):

```c
if (DAT_00402138 == '+') {
    uValue = FUN_004015b6((char)uValue,
                          extraout_DL_02,
                          extraout_CL_02,
                          DAT_00402120);
} else if (DAT_00402138 == '-') {
    uValue = FUN_004015c4((char)uValue,
                          extraout_DL_02,
                          extraout_CL_02,
                          (char)DAT_00402120,
                          uValue);
} else if (DAT_00402138 == '*') {
    uValue = FUN_004015d4();
} else if (DAT_00402138 == '/') {
    uValue = FUN_004015e4();
}
```

So I renamed those functions to `op_plus`, `op_minus`, `op_times` and `op_divide`. They all look something like this:

```
004015b6 c8 00 00 00     ENTER      0x0,0x0
004015ba 8b 45 08        MOV        param_1,dword ptr [EBP + param_4]
004015bd 90              NOP
004015be 90              NOP
004015bf 90              NOP
004015c0 c9              LEAVE
004015c1 c2 08 00        RET        0x8
```

It looks like we're supposed to patch the binary so the calculator works and the flag is shown. Unfortunately, Ghidra [can't save patched files](https://github.com/NationalSecurityAgency/ghidra/issues/19) (unless they're opened as raw binary, which seems to disable many useful analysis features). I found a [SavePatch.py script](https://github.com/schlafwandler/ghidra_SavePatch) which seems to work fine.

However, I was a bit confused about the calling convention at that point - where do I get the arguments, which registers can I use, and how do I need to return the result?

I decided to dig a bit deeper - looking at references, we also find a function I call `show_flag` at `00401519`:

```
ENTER      0x0,0x0

PUSH       0x1762a070
PUSH       0x21ceb5d8
CALL       op_plus
MOV        [DAT_004020a0],param_1

PUSH       0x38b57698
PUSH       0xaae5b913
CALL       op_minus
MOV        [DAT_004020a4],param_1

PUSH       0x2
PUSH       0xbec8cad6
CALL       op_divide
MOV        [DAT_004020a8],param_1

PUSH       0x2
PUSH       0x33b0b623
CALL       op_times
MOV        [DAT_004020ac],param_1

PUSH       0x53bd761a
PUSH       0x18a3cd45
CALL       op_plus
MOV        [DAT_004020b0],param_1

PUSH       0x46c920f4
PUSH       0xa8359657
CALL       op_minus
MOV        [DAT_004020b4],param_1

PUSH       0x4
PUSH       0x1f5c8c1d
CALL       op_times
MOV        [DAT_004020b8],param_1

PUSH       DAT_004020a0
PUSH       0x3e8
PUSH       dword ptr [EBP + param_4]
CALL       SetDlgItemTextA

LEAVE
RET        0x4
```

Since those oparations seemed quite straightforward, I thought I'd instead reimplement them in Python instead.

If we look at the displayed flag, it starts with `ØµÎ!`. Encoded as [Latin1](https://de.wikipedia.org/wiki/ISO_8859-1) that'd be: 0xD8, 0xB5, 0xCE, 0x21

This matches up with the `0x21ceb5d8` above after some bit-twiddling. Let's write a decoding function:

In [11]:
def decode(dword):
    data = dword.to_bytes(4, byteorder='little')
    return data.decode('latin1')

print(decode(0x21ceb5d8))

ØµÎ!


Then we can get the flag by reimplementing the same calculations in Python:

In [14]:
flag = decode(0x21ceb5d8 + 0x1762a070)
flag += decode(0xaae5b913 - 0x38b57698)
flag += decode(0xbec8cad6 // 2)
flag += decode(0x33b0b623 * 2)
flag += decode(0x53bd761a + 0x18a3cd45)
flag += decode(0xa8359657 - 0x46c920f4)
flag += decode(0x1f5c8c1d * 4)
print(flag)

HV19{B0rked_Flag_Calculat0r}
